In [1]:
import pandas as pd
import itertools
import sys
import os
from pathlib import Path
import pickle
import pandas as pd
import json
import openai
import pickle
import dotenv
import random
import time
path = Path(os.getcwd())
sys.path.append(str(path.parent.absolute()))
import propmts
from common_utils import *
from factual_recall_qa.utils import *
from constants import *



In [2]:
setup_directories()
with open('../factual_recall_qa/dataset/icl_query_year_nobel_laureates.pickle', 'rb') as handle:
    df = pickle.load(handle)
with open('../knowledge_validation/dataset/nobel_laureates_attribute_verified.pickle', 'rb') as handle:
    data = pickle.load(handle)

In [3]:
meta = metadata[NOBEL_LAUREATES]
for concept_class in meta[TARGET_ATTRIBUTES].keys():
    for attribute in meta[TARGET_ATTRIBUTES][concept_class]:
        filtered_df = df[df[TARGET_ATTRIBUTES]==attribute]
        if len(filtered_df)==0:
            continue
        correct  = filtered_df.apply(lambda x: score_attribute(concept_class, x[GROUND_TRUTH], x[GPT_4_RESPONSE]), axis=1)
        print(f"{concept_class} {attribute} {len(filtered_df)} Number of correct: {sum(correct)} Number of Hallucinated: {str(len(correct) - sum(correct))}")

year born 500 Number of correct: 460 Number of Hallucinated: 40
year died 499 Number of correct: 4 Number of Hallucinated: 495
year year 500 Number of correct: 498 Number of Hallucinated: 2


In [7]:
result_entries = df.to_dict('records')
for e in range(len(result_entries)):
    string = result_entries[e][ICL_PROMPT_COLUMN]
    record_found = None
    k =0
    for c in data:
        suffix =[ 
            c[CONTEXTUALISING_ATTRIBUTES][Attribute.FIRST_NAME.value],
        ]
        if c[CONTEXTUALISING_ATTRIBUTES][Attribute.SURNAME.value] is not None:
            suffix.append(c[CONTEXTUALISING_ATTRIBUTES][Attribute.SURNAME.value])
        if c[CONTEXTUALISING_ATTRIBUTES][Attribute.MOTIVATION_NOBEL.value] is not None:
            suffix.append(c[CONTEXTUALISING_ATTRIBUTES][Attribute.MOTIVATION_NOBEL.value])
        if c[CONTEXTUALISING_ATTRIBUTES][Attribute.CATEGORY_NOBEL.value] is not None:
            suffix.append(c[CONTEXTUALISING_ATTRIBUTES][Attribute.CATEGORY_NOBEL.value])
        suffix = ', '.join(suffix) + ': '
        if string.endswith(suffix):
            record_found = c
            break
    if record_found is None:
        print(string)
        print(result_entries[e][ICL_PROMPT_COLUMN])
    result_entries[e][Attribute.BIRTH_DATE_NOBEL.value] = extract_year(record_found[TARGET_ATTRIBUTES][ConceptClass.YEAR.value][Attribute.BIRTH_DATE_NOBEL.value])
    result_entries[e][Attribute.DEATH_DATE_NOBEL.value] = extract_year(record_found[TARGET_ATTRIBUTES][ConceptClass.YEAR.value][Attribute.DEATH_DATE_NOBEL.value])
    result_entries[e][Attribute.YEAR.value] = extract_year(record_found[TARGET_ATTRIBUTES][ConceptClass.YEAR.value][Attribute.YEAR.value])
    # result_entries[e][Attribute.BIRTH_CITY.value] = record_found[TARGET_ATTRIBUTES][ConceptClass.PLACE.value][Attribute.BIRTH_CITY.value]
    # result_entries[e][Attribute.DEATH_CITY.value] = record_found[TARGET_ATTRIBUTES][ConceptClass.PLACE.value][Attribute.DEATH_CITY.value]
    # result_entries[e][Attribute.WORK_CITY.value] = record_found[TARGET_ATTRIBUTES][ConceptClass.PLACE.value][Attribute.WORK_CITY.value]
    

In [8]:
df = pd.DataFrame(result_entries)

In [9]:
df.to_csv('dataset/icl_query_response_nobel_laureates_year.csv')

In [10]:
df

,icl_prompt,ground_truth,target_attributes,GPT 4 Response,born,died,year
0,"Akira, Suzuki, ""for palladium-catalyzed cross ...",1917,born,1917,1917,1985,1972
1,"Vitaly L., Ginzburg, ""for pioneering contribut...",1913,born,1913,1913,2005,1985
2,"Werner, Forssmann, ""for their discoveries conc...",1946,born,1946,1946,2016,1999
3,"Vicente, Aleixandre, ""for a creative poetic wr...",1920,born,1920,1920,2017,1981
4,"Vincent, du Vigneaud, ""for his work on biochem...",1905,born,1905,1905,1984,1965
...,...,...,...,...,...,...,...
1494,"George, Wald, ""for their discoveries concernin...",1977,year,1977,1917,2003,1977
1495,"Thomas C., Südhof, ""for their discoveries of m...",1953,year,1953,1881,1965,1953
1496,"Frederick, Sanger, ""for their contributions co...",2006,year,2006,1959,None,2006
1497,"Sir Frank Macfarlane, Burnet, ""for discovery o...",1922,year,1922,1861,1930,1922


In [ ]:
for i in data:
    if i[CONTEXTUALISING_ATTRIBUTES][Attribute.FIRST_NAME.value] in  :